In [3]:
import os

# Read the API key from secrets.txt
with open('../secrets/secrets.txt', 'r') as file:
    api_key = file.readline().strip()

os.environ["OPENAI_API_KEY"] = api_key

In [4]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader

In [5]:
from langchain.document_loaders import CSVLoader

# Initialize the CSVLoader with the path to your CSV file
loader = CSVLoader(file_path='../data/data_img_str_url.csv',metadata_columns=['images','article_url','title'])

# Load the documents
documents = loader.load()

# Print the loaded documents
for doc in documents:
    print(doc)
    break

page_content='title: Reinforcement Learning Heats Up
content: Reinforcement learning is emerging as an avenue for building large language models with advanced reasoning capabilities.
What’s new:Two recent high-performance models,DeepSeek-R1(and its variants including DeepSeek-R1-Zero) andKimi k1.5, learned to improve their generated lines of reasoning via reinforcement learning.o1pioneered this approach last year.
Reinforcement learning (RL) basics:RL rewards or punishes a model for performing particular actions or achieving certain objectives. Unlike supervised and unsupervised learning, which compare the model's output to a known ground truth, RL doesn’t explicitly tell a model what it should output. Instead, the model starts out behaving randomly and discovers desired behaviors by earning rewards for its actions. This makes RL especially popular for training machine learning models that play games or control robots.
How it works:To improve thechain of thought(CoT) generated by a lar

In [6]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [7]:
len(texts)

1495

In [8]:
texts[3]

Document(metadata={'source': 'data_img_str_url.csv', 'row': 0, 'images': 'https://dl-staging-website.ghost.io/content/images/2025/01/unnamed--46-.gif', 'article_url': 'https://www.deeplearning.ai/the-batch/issue-286/'}, page_content='We’re thinking:Less than three years ago, reinforcement learning looked toofinickyto be worth the trouble. Now it’s a key direction in language modeling. Machine learning continues to be full of surprising twists!')

In [6]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = '../db'

## here we are using OpenAI embeddings but in future we will swap out to local embeddings
embedding = OpenAIEmbeddings()



/tmp/ipykernel_69659/653128838.py:6: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()


In [ ]:
vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [22]:
# persiste the db to disk
vectordb.persist()
vectordb = None

/tmp/ipykernel_3001307/2083128404.py:2: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [7]:
# Now we can load the persisted database from disk, and use it as normal. 
vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding)

/tmp/ipykernel_69659/3063115583.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=persist_directory,


In [8]:
retriever = vectordb.as_retriever()

In [9]:
docs_retrieved = retriever.get_relevant_documents("What is deepseek?")

/tmp/ipykernel_69659/2591566301.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs_retrieved = retriever.get_relevant_documents("What is deepseek?")


In [10]:
docs_retrieved[1]

Document(metadata={'article_url': 'https://www.deeplearning.ai/the-batch/issue-285/', 'images': 'https://dl-staging-website.ghost.io/content/images/2025/01/unnamed--45-.gif', 'row': 3, 'source': 'data_img_str_url.csv'}, page_content='title: DeepSeek Sharpens Its Reasoning\ncontent: A new open model rivals OpenAI’s o1, and it’s free to use or modify.\nWhat’s new:DeepSeek releasedDeepSeek-R1, a large language model that executes long lines of reasoning before producing output. The code and weights arelicensedfreely for commercial and personal use, including training new models on R1 outputs. Thepaperprovides an up-close look at the training of a high-performance model that implements a chain of thought without explicit prompting. (DeepSeek-R1-lite-previewcame out in November with fewer parameters and a different base model.)')

In [11]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [12]:
# create the chain to answer questions 
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(), 
                                  chain_type="stuff", 
                                  retriever=retriever, 
                                  return_source_documents=True)

/tmp/ipykernel_69659/1952940232.py:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),


In [13]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'], source.metadata['row'], source.metadata['article_url'])

In [14]:
# full example
query = "Who is Mustafa Suleyman?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

/tmp/ipykernel_69659/3627713110.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm_response = qa_chain(query)



Mustafa Suleyman is the Chief Executive Officer of Microsoft AI and the co-founder of Inflection AI and founder of DeepMind Technologies.


Sources:
data_img_str_url.csv 17 https://www.deeplearning.ai/the-batch/issue-282/
data_img_str_url.csv 17 https://www.deeplearning.ai/the-batch/issue-282/
data_img_str_url.csv 73 https://www.deeplearning.ai/the-batch/issue-265/


In [15]:
llm_response["source_documents"][0].metadata

{'article_url': 'https://www.deeplearning.ai/the-batch/issue-282/',
 'images': 'https://dl-staging-website.ghost.io/content/images/2025/01/unnamed--41-.png',
 'row': 17,
 'source': 'data_img_str_url.csv'}